<a href="https://colab.research.google.com/github/abhi1021/resnet50-imagenet-1k/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training ImageNette Model (Google Colab)
This notebook trains the ResNet-50 model on ImageNette-160 (10 classes, epochs=3)

## Download ImageNette Dataset

In [1]:
# Clone the repository
!git clone https://github.com/abhi1021/resnet50-imagenet-1k.git

Cloning into 'resnet50-imagenet-1k'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 98 (delta 47), reused 80 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 380.55 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [2]:
# Navigate to the repository directory
%cd resnet50-imagenet-1k

/content/resnet50-imagenet-1k


In [3]:
# Install required dependencies
!pip install -q -r requirements.txt
print("✓ Dependencies installed successfully")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✓ Dependencies installed successfully


In [4]:
# Download and extract ImageNette dataset
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
!tar -xzf imagenette2-160.tgz
!ls -la imagenette2-160

--2025-10-18 13:52:52--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.180.29, 52.216.220.88, 52.216.52.240, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.180.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99003388 (94M) [application/x-tar]
Saving to: ‘imagenette2-160.tgz’

imagenette2-160.tgz 100%[===================>]  94.42M  15.0MB/s    in 8.1s    

2025-10-18 13:53:02 (11.6 MB/s) - ‘imagenette2-160.tgz’ saved [99003388/99003388]

total 1228
drwxrwxr-x  4  501 staff    4096 Feb  6  2021 .
drwxr-xr-x  5 root root     4096 Oct 18 13:53 ..
-rw-r--r--  1  501 staff    6148 Feb  6  2021 .DS_Store
-rw-r--r--  1  501 staff 1230799 Feb  6  2021 noisy_imagenette.csv
drwxr-xr-x 12  501 staff    4096 Oct 18 13:53 train
drwxr-xr-x 12  501 staff    4096 Nov 30  2019 val


## Verify Dataset Structure

In [5]:
# Check dataset structure (should have train and val folders)
!ls -la imagenette2-160/train | head -15
!echo "\n"
!ls -la imagenette2-160/val | head -15

total 388
drwxr-xr-x 12 501 staff  4096 Oct 18 13:53 .
drwxrwxr-x  4 501 staff  4096 Feb  6  2021 ..
-rw-r--r--  1 501 staff  6148 Feb  5  2021 .DS_Store
drwxr-xr-x  2 501 staff 40960 Oct 18 13:53 n01440764
drwxr-xr-x  2 501 staff 36864 Oct 18 13:53 n02102040
drwxr-xr-x  2 501 staff 45056 Nov 30  2019 n02979186
drwxr-xr-x  2 501 staff 36864 Oct 18 13:53 n03000684
drwxr-xr-x  2 501 staff 36864 Oct 18 13:53 n03028079
drwxr-xr-x  2 501 staff 36864 Oct 18 13:53 n03394916
drwxr-xr-x  2 501 staff 36864 Oct 18 13:53 n03417042
drwxr-xr-x  2 501 staff 36864 Oct 18 13:53 n03425413
drwxr-xr-x  2 501 staff 36864 Oct 18 13:53 n03445777
drwxr-xr-x  2 501 staff 36864 Oct 18 13:53 n03888257
\n
total 208
drwxr-xr-x 12 501 staff  4096 Nov 30  2019 .
drwxrwxr-x  4 501 staff  4096 Feb  6  2021 ..
drwxr-xr-x  2 501 staff 20480 Oct 18 13:53 n01440764
drwxr-xr-x  2 501 staff 20480 Oct 18 13:53 n02102040
drwxr-xr-x  2 501 staff 20480 Nov 30  2019 n02979186
drwxr-xr-x  2 501 staff 20480 Oct 18 13:53 n03000684


## Import Libraries and Check GPU

In [6]:
import torch
import sys
from train import ImageClassificationTrainer

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
    print(f"Memory Cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")
else:
    print("WARNING: Running on CPU. Training will be slower.")
    print("In Colab, go to Runtime > Change runtime type > Hardware accelerator > GPU")

Using device: cuda
GPU: Tesla T4
Memory Allocated: 0.00 MB
Memory Cached: 0.00 MB


## Initialize and Train ImageNette Model with ResNet-50

In [7]:
# Create trainer with specified settings
# Configuration:
#   - Model: ResNet-50 (25.6M parameters)
#   - Dataset: ImageNette (10 classes, 160x160 images)
#   - Epochs: 3
#   - Batch Size: 128 (adjusted for Colab GPU)
#   - Learning Rate: Warmup (0.01→0.1) + Cosine annealing
#   - MixUp augmentation: Enabled (alpha=0.2)
#   - Label smoothing: 0.1
#   - Mixed precision: Enabled
#   - Gradient clipping: 1.0
trainer = ImageClassificationTrainer(
    model_name='resnet50',
    dataset='imagenet',
    data_dir='imagenette2-160',
    epochs=3,
    batch_size=128,
    num_workers=2
)

print("Starting ImageNette Model Training with ResNet-50")
print("="*70)
print(f"Configuration:")
print(f"  - Model: ResNet-50 (25.6M parameters)")
print(f"  - Dataset: ImageNette (10 classes)")
print(f"  - Image Size: 160x160 (resized to 224x224)")
print(f"  - Batch Size: 128")
print(f"  - Epochs: 3")
print(f"  - Optimizer: SGD (momentum=0.9, weight_decay=1e-3)")
print(f"  - Learning Rate: Warmup (0.01→0.1) + Cosine annealing")
print(f"  - MixUp: Enabled (alpha=0.2)")
print(f"  - Label Smoothing: 0.1")
print(f"  - Mixed Precision: Enabled")
print(f"  - Gradient Clipping: 1.0")
print("="*70)

✓ Using NVIDIA GPU: Tesla T4

Loading ImageNet dataset from: imagenette2-160
ImageNet dataset loaded:
  Training samples: 9,469
  Validation samples: 3,925
  Number of classes: 10
Detected 10 classes in dataset
Starting ImageNette Model Training with ResNet-50
Configuration:
  - Model: ResNet-50 (25.6M parameters)
  - Dataset: ImageNette (10 classes)
  - Image Size: 160x160 (resized to 224x224)
  - Batch Size: 128
  - Epochs: 3
  - Optimizer: SGD (momentum=0.9, weight_decay=1e-3)
  - Learning Rate: Warmup (0.01→0.1) + Cosine annealing
  - MixUp: Enabled (alpha=0.2)
  - Label Smoothing: 0.1
  - Mixed Precision: Enabled
  - Gradient Clipping: 1.0


In [8]:
# Run training
final_accuracy = trainer.run()


Training resnet50 for 3 epochs on ImageNet (10 classes)

Model Architecture Summary
Device: cuda
Model: resnet50
Number of classes: 10
Mixed Precision: True
MixUp: True (alpha=0.2)
Label Smoothing: 0.1

Model Summary:



Epoch 1 Loss=2.3117 Acc=10.46% LR=0.027757: 100%|██████████| 74/74 [00:51<00:00,  1.43it/s]



Test set: Average loss: 2.4949, Accuracy: 391/3925 (9.96%)

*** New best model! Test Accuracy: 9.96% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 9.96%



Epoch 2 Loss=2.3499 Acc=11.65% LR=0.045757: 100%|██████████| 74/74 [00:49<00:00,  1.48it/s]



Test set: Average loss: 2.3777, Accuracy: 583/3925 (14.85%)

*** New best model! Test Accuracy: 14.85% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 14.85%



Epoch 3 Loss=2.3689 Acc=16.48% LR=0.063757: 100%|██████████| 74/74 [00:49<00:00,  1.50it/s]



Test set: Average loss: 2.3256, Accuracy: 739/3925 (18.83%)

*** New best model! Test Accuracy: 18.83% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 18.83%


📦 Saving final model...
✓ Checkpoint saved: final_model.pth

Training completed. Best test accuracy: 18.83%

TRAINING AND TESTING LOSS
   (Y)     ^
2.74435286 |
2.60713522 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2.46991757 | ⠤⠤⠤⠤⢄⣀⣀⣀⣀⣀⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2.33269993 | ⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠬⠭⠭⠭⠭⠭⠭⠭⠭⠶⠶⠶⠶⠶⠶⠶⠶⠶⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀
2.19548229 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2.05826464 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
1.92104700 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
1.78382936 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
1.64661172 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

## Training Summary

In [9]:
print("\n" + "="*70)
print("TRAINING COMPLETED")
print("="*70)
print(f"Model: ResNet-50")
print(f"Dataset: ImageNette (10 classes)")
print(f"Final Best Test Accuracy: {final_accuracy:.2f}%")
print("="*70)
print("\nCheckpoints saved in ./checkpoints/")
print("  - best_model.pth (highest test accuracy)")
print("  - final_model.pth (final epoch)")
print("  - training_curves.png (visualization)")
print("  - metrics.json (complete training history)")


TRAINING COMPLETED
Model: ResNet-50
Dataset: ImageNette (10 classes)
Final Best Test Accuracy: 18.83%

Checkpoints saved in ./checkpoints/
  - best_model.pth (highest test accuracy)
  - final_model.pth (final epoch)
  - training_curves.png (visualization)
  - metrics.json (complete training history)
